# Logistic Regression - Raw Implementation

## Dataset: Amazon Reviews

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression


products = pd.read_csv('amazon_baby_subset.csv')

print("Name of the first 10 products:\n {}".format(products['name'][:10]))

print("number of positive reviews is: {}".format(np.sum(products['sentiment'] > 0) / len(products)))
print("number of negative reviews is: {}".format(np.sum(products['sentiment'] < 0) / len(products)))

Name of the first 10 products:
 0    Stop Pacifier Sucking without tears with Thumb...
1      Nature's Lullabies Second Year Sticker Calendar
2      Nature's Lullabies Second Year Sticker Calendar
3                          Lamaze Peekaboo, I Love You
4    SoftPlay Peek-A-Boo Where's Elmo A Children's ...
5                            Our Baby Girl Memory Book
6    Hunnt&reg; Falling Flowers and Birds Kids Nurs...
7    Blessed By Pope Benedict XVI Divine Mercy Full...
8    Cloth Diaper Pins Stainless Steel Traditional ...
9    Cloth Diaper Pins Stainless Steel Traditional ...
Name: name, dtype: object
number of positive reviews is: 0.500810220078384
number of negative reviews is: 0.4991897799216159


### Preprocessing

In [2]:
import json

with open("important_words.json") as json_file:
    important_words = json.load(json_file)
    
products = products.fillna({'review':''})  # fill in N/A's in the review column

import string

def remove_punctuation(text):
    remove_punct_map = dict.fromkeys(map(ord, string.punctuation))
    return text.translate(remove_punct_map)


products['review_clean'] = products['review'].apply(remove_punctuation)


for word in important_words:
    products[word] = products['review_clean'].apply(lambda s : s.split().count(word))

products['contains_perfect'] = products['perfect'].apply(lambda k: 1 if k >=1 else 0)

products.head()

,name,review,rating,sentiment,review_clean,baby,one,great,love,use,...,picture,completely,wish,buying,babies,won,tub,almost,either,contains_perfect
0,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1,All of my kids have cried nonstop when I tried...,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Nature's Lullabies Second Year Sticker Calendar,We wanted to get something to keep track of ou...,5,1,We wanted to get something to keep track of ou...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Nature's Lullabies Second Year Sticker Calendar,My daughter had her 1st baby over a year ago. ...,5,1,My daughter had her 1st baby over a year ago S...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Lamaze Peekaboo, I Love You","One of baby's first and favorite books, and it...",4,1,One of babys first and favorite books and it i...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,SoftPlay Peek-A-Boo Where's Elmo A Children's ...,Very cute interactive book! My son loves this ...,5,1,Very cute interactive book My son loves this b...,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


In [3]:
print(products.shape)

(53072, 199)


### Create numpy array from df

In [4]:
def get_numpy_data(dataframe, features, label):
    dataframe['constant'] = 1
    features = ['constant'] + features
    features_frame = dataframe[features]
    feature_matrix = features_frame.as_matrix()  # old, use .values instead
    label_sarray = dataframe[label]
    label_array = label_sarray.as_matrix()  #.values instead
    return(feature_matrix, label_array)

feature_matrix, label_array = get_numpy_data(products, important_words, 'sentiment')


C:\Users\abido\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """
C:\Users\abido\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [5]:
feature_matrix.shape

(53072, 194)

## Model

In [6]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def predict_probability(feature_matrix, coefficients):
    # Take dot product of feature_matrix and coefficients  
    # YOUR CODE HERE
    score = np.dot(feature_matrix, coefficients)
    
    # Compute P(y_i = +1 | x_i, w) using the link function
    # YOUR CODE HERE
    predictions = sigmoid(score)
    
    # return predictions
    return predictions


def feature_derivative(errors, feature):     
    # Compute the dot product of errors and feature
    derivative = np.dot(errors, feature)
        # Return the derivative
    return derivative 

def compute_log_likelihood(feature_matrix, sentiment, coefficients):
    indicator = (sentiment==+1)
    scores = np.dot(feature_matrix, coefficients)
    lp = np.sum((indicator-1)*scores - np.log(1. + np.exp(-scores)))
    return lp

from math import sqrt
def logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter):
    coefficients = np.array(initial_coefficients) # make sure it's a numpy array
    for itr in range(max_iter):
        # Predict P(y_i = +1|x_1,w) using your predict_probability() function
        # YOUR CODE HERE
        predictions = predict_probability(feature_matrix, coefficients)

        # Compute indicator value for (y_i = +1)
        indicator = (sentiment==+1)

        # Compute the errors as indicator - predictions
        errors = indicator - predictions

        for j in range(len(coefficients)): # loop over each coefficient
            # Recall that feature_matrix[:,j] is the feature column associated with coefficients[j]
            # compute the derivative for coefficients[j]. Save it in a variable called derivative
            # YOUR CODE HERE
            derivative = feature_derivative(errors, feature_matrix[:,j])
            

            # add the step size times the derivative to the current coefficient
            # YOUR CODE HERE
            coefficients[j] = coefficients[j] + step_size * derivative

        # Checking whether log likelihood is increasing
        if itr <= 15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) \
        or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:
            lp = compute_log_likelihood(feature_matrix, sentiment, coefficients)
            print('iteration %*d: log likelihood of observed labels = %.8f' % \
                (int(np.ceil(np.log10(max_iter))), itr, lp))
    return coefficients

### Training and Evaluation

In [7]:
coefs = logistic_regression(feature_matrix=feature_matrix, sentiment=label_array, initial_coefficients = np.zeros(194), step_size=1e-7, max_iter=301)

iteration   0: log likelihood of observed labels = -36780.91768478
iteration   1: log likelihood of observed labels = -36775.13434712
iteration   2: log likelihood of observed labels = -36769.35713564
iteration   3: log likelihood of observed labels = -36763.58603240
iteration   4: log likelihood of observed labels = -36757.82101962
iteration   5: log likelihood of observed labels = -36752.06207964
iteration   6: log likelihood of observed labels = -36746.30919497
iteration   7: log likelihood of observed labels = -36740.56234821
iteration   8: log likelihood of observed labels = -36734.82152213
iteration   9: log likelihood of observed labels = -36729.08669961
iteration  10: log likelihood of observed labels = -36723.35786366
iteration  11: log likelihood of observed labels = -36717.63499744
iteration  12: log likelihood of observed labels = -36711.91808422
iteration  13: log likelihood of observed labels = -36706.20710739
iteration  14: log likelihood of observed labels = -36700.5020

In [8]:
def predict(feature_matrix, coefficients):
    scores = np.dot(feature_matrix, coefs)
    preds = scores > 0
    return preds

preds = predict(feature_matrix, coefs)


In [9]:
# num of reviews predicted to have positive sentiment
np.sum(preds == 1)

25126

In [10]:
# Accuracy
np.sum(np.equal(preds,label_array>0)) / len(label_array)

0.7518653904130238

In [11]:
coefficients = list(coefs[1:]) # exclude intercept
word_coefficient_tuples = [(word, coefficient) for word, coefficient in zip(important_words, coefficients)]
word_coefficient_tuples = sorted(word_coefficient_tuples, key=lambda x:x[1], reverse=True)
word_coefficient_tuples

[('great', 0.0665460841704577),
 ('love', 0.06589076292212326),
 ('easy', 0.06479458680257838),
 ('little', 0.04543562630842137),
 ('loves', 0.04497640139490604),
 ('well', 0.030135001092107067),
 ('perfect', 0.02973993710496846),
 ('old', 0.020077541034775378),
 ('nice', 0.018408707995268996),
 ('daughter', 0.01770319990570169),
 ('soft', 0.01757027224560289),
 ('fits', 0.01688247107140872),
 ('happy', 0.016805295889768077),
 ('baby', 0.015565696580423507),
 ('recommend', 0.015408450108008665),
 ('also', 0.015216196422918842),
 ('best', 0.014991791565630266),
 ('comfortable', 0.0132539900815849),
 ('car', 0.012685935745813379),
 ('clean', 0.012018174433365525),
 ('son', 0.011944817713693953),
 ('bit', 0.011708248093123262),
 ('works', 0.011703160621987424),
 ('size', 0.010715966516270298),
 ('stroller', 0.009909164635972734),
 ('room', 0.009783241021568061),
 ('price', 0.00957273354359018),
 ('play', 0.00917842898398431),
 ('easily', 0.009032818138954405),
 ('kids', 0.0085828430043465